작업형 2
- X_train = 구매속성
- y_train = 성별 (0:여자, 1:남자)
- X_test = cust_id,구매속성 -> 성별예측 

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
# 1. 데이터불러오기
import pandas as pd
X_train = pd.read_csv("X_train.csv", encoding = 'euc-kr')
y_train = pd.read_csv("y_train.csv")
test = pd.read_csv("X_test.csv",encoding = 'euc-kr')
cust_id = test.cust_id
test = test.drop(['cust_id'],axis=1)

In [14]:
# 2. 데이터합치기
trainDF = pd.merge(X_train, y_train)

In [15]:
# 3. 결측치 확인 
trainDF.isnull().sum()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
gender        0
dtype: int64

In [16]:
# 4. 환불금액 살펴보기
pd.DataFrame(trainDF['환불금액'])

,환불금액
0,6860000.0
1,300000.0
2,NaN
3,NaN
4,NaN
...,...
3495,NaN
3496,6049600.0
3497,NaN
3498,NaN


In [17]:
# 5. 환불금액 NAN -> 0으로 결측치 채우기 
trainDF['환불금액'] = trainDF['환불금액'].fillna(0)
# test 도 챙겨주자!
test['환불금액'] = test['환불금액'].fillna(0)

In [18]:
# 6. 주구매상품, 주구매지점 범주형 변수 -> LabelEncoder 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
trainDF.loc[:,['주구매상품','주구매지점']] = trainDF.loc[:,['주구매상품','주구매지점']].apply(le.fit_transform)
# test 도 챙겨주자! 
test.loc[:,['주구매상품','주구매지점']] = test.loc[:,['주구매상품','주구매지점']].apply(le.fit_transform)

In [19]:
# 7. 합친 데이터를 X, y로 다시 나누자
X = trainDF.drop(['cust_id','gender'], axis=1) # axis=1 안해주면 에러남. 열(column)을 드랍한다고 알려주기
y = pd.DataFrame(trainDF.gender) # series로 되는게 싫어서 애초에 데이터 프레임으로 바꿈 

In [20]:
# 8. train test 나눠주자 (? 원래 있던 X_test는 마지막에 어떻게 검증하는지 모르겠음)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state=42)

In [21]:
# 9. 정규화 - MinMax/StandardScaler 중 선택 둘다해서 결과비교해보기
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)
test_scaled = scaler.transform(test)

In [22]:
# 10. 모델적용 LogisticRegression 분류문제니까
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)

In [24]:
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("오차행렬 train data \n", confusion_train)

오차행렬 train data 
 [[1540   98]
 [ 859  128]]


In [38]:
# X_scaled_test, y_test 챙겨주자
from sklearn.metrics import accuracy_score
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)
accuracy = accuracy_score(y_test, pred_test)
print('로지스틱회귀모델: {:.4f}'.format(accuracy))

로지스틱회귀모델: 0.6331


In [39]:
confusion_test = confusion_matrix(y_test, pred_test)
print("오차행렬 test data \n", confusion_test)

오차행렬 test data 
 [[517  29]
 [292  37]]


In [40]:
from sklearn.metrics import roc_auc_score ,roc_curve

lr_probs = model.predict_proba(X_scaled_test)
lr_auc = roc_auc_score(y_test, lr_probs[:,1])
lr_auc

0.5954273689835999

In [41]:
# 결과를 보면 train 데이터에 과적합 되었다는걸 알 수 있다.
#test = test.drop(['cust_id'],axis=1)
predict_gender = pd.DataFrame(model.predict_proba(test_scaled)[:,1], columns=['gender'])
predict_gender

,gender
0,0.502881
1,0.210969
2,0.132289
3,0.348936
4,0.397341
...,...
2477,0.489339
2478,0.394878
2479,0.438478
2480,0.389916


In [43]:
# 제출할 데이터 합치기 
result = pd.concat([cust_id,predict_gender], axis=1)
result.head()

,cust_id,gender
0,3500,0.502881
1,3501,0.210969
2,3502,0.132289
3,3503,0.348936
4,3504,0.397341


In [111]:
pd.read_csv("git/DataAnalysis/RF.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'git/DataAnalysis/RF.csv'

In [97]:
type(predict_gender)

numpy.ndarray